#### Loading libraries

In [1]:
# Load tensorflow
import tensorflow as tf
# Below command is to avoid the known bug which prevents computation on some GPU devices
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# Load preprocessing tools
from scipy.ndimage.filters import gaussian_filter
from tensorflow.keras.utils import to_categorical
from PIL import Image
# Load model building blocks
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
# Load pre-trained model library
from tensorflow.keras import applications
# Load miscelaneous libraries
import numpy as np
import os
import matplotlib.pylab as plt
import gc
# Settings to set the max output of numpy to infty
# import sys
# np.set_printoptions(threshold=sys.maxsize)

#### Define image processor and data generator

In [2]:
# Define gaussian blur class
class GaussBlur:
    def __init__(self, radius):
        self.radius = radius
    def blur(self, image):
        return gaussian_filter(image, sigma = self.radius)

In [3]:
# Define class for data generation
class DataGenerator:
    'Generates data for Keras'
    def __init__(self, list_image_paths = None,  
                 dim = None,
                 n_channels = 3, 
                 rescale = 1, 
                 preprocessing_func = None):
        'Initialization'
        self.list_image_paths = list_image_paths
        self.dim = dim
        self.n_channels = n_channels
        self.rescale = rescale
        self.preprocessing_func = preprocessing_func

    def generate(self):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        
        # Initialisation
        X = np.empty((len(self.list_image_paths), *self.dim, self.n_channels))
        
        # Generate data
        for i, image_path in enumerate(self.list_image_paths):
            # Load image and transform
            image = Image.open(os.path.join(image_path))
            if self.dim is not None:
                image = image.resize(self.dim, resample = Image.NEAREST)
            image = np.array(image)[:, :, :self.n_channels]
            image = image * self.rescale
            if self.preprocessing_func is not None:
                image = self.preprocessing_func(image)
            # Store sample
            X[i,] = image

        return X

In [4]:
# Define algorithm for reordering the data based on the index array
def reorder(ind, arr):
    new_arr = np.zeros(arr.shape)
    for i in range(len(arr)):
        new_arr[i] = arr[ind[i]]
    return new_arr  

def order_labels(y_lai, y_lm, y_lat):
    y_lm_reord = dict()
    y_lai_reord = dict()
    y_lat_reord = dict()
    # Perform reording
    for k in range(5):
        i = k + 2
        y_lai_reord[i] = np.sort(y_lai[i], axis = -1) 
        ind_matrix = np.argsort(y_lai[i], axis = -1)
        y_lm_reord[i] = np.array([reorder(ind_matrix[j,:], y_lm[i][j,:]) for j in range(y_lm[i].shape[0])])
        y_lat_reord[i] = np.array([reorder(ind_matrix[j,:], y_lat[i][j,:]) for j in range(y_lat[i].shape[0])])
    return y_lai_reord, y_lm_reord, y_lat_reord

In [5]:
# Define sorter of image names in order by image number (default is alphanumric)
def sorter(item):
    # Since highest marks first, least error = most marks
    radius = float(item[1 : item.find('_')])
    num_img = int(item[item.find('g') + 1 : item.find('j') - 1])
    return (radius, num_img)

#### Load the data

In [6]:
# Extract image paths
img_paths = {}
for k in range(5):
    i = k + 2
    path_prefix = os.path.join(os.getcwd(), 'image_data', 'test', str(i))
    img_paths[i] = [os.path.join(path_prefix, name) for name in sorted(os.listdir(path_prefix), key = sorter)]

# Apply data generators
gaussblur = GaussBlur(1)
params = {'dim': (128, 128), 
          'n_channels': 3, 
          'rescale': 1 / 255, 
          'preprocessing_func': gaussblur.blur
          }
test_generator = {}
for k in range(5):
    i = k + 2
    test_generator[i] = DataGenerator(img_paths[i], **params)
    
# Load images (for all number of forces) as matrix
X = dict()
for k in range(5):
    i = k + 2
    X[i] = test_generator[i].generate()

In [7]:
# Load angle labels
path_prefix = os.path.join(os.getcwd(), 'labels', 'test')
y_lai = dict()
for k in range(5):
    i = k + 2
    path = os.path.join(path_prefix, str(i))
    y_lai[i] = np.load(os.path.join(path,'mags.npy'))

In [8]:
# Load models
models_path = os.path.join(os.getcwd(), 'models')

model_class = load_model(os.path.join(models_path, 'xception_num_forces.h5'))                       

In [9]:
# predict class
predicted_class = dict()
indices_correct_class = dict()
indices_incorrect_class = dict()
predicted_incorrect_class = dict()
for k in range(5):
    i = k + 2
    predicted_class[i] = model_class.predict(X[i]).argmax(-1)+2
    indices_correct_class[i] = np.argwhere(predicted_class[i] == i).flatten().tolist()
    indices_incorrect_class[i] = np.argwhere(predicted_class[i] != i).flatten().tolist()
    predicted_incorrect_class[i] = predicted_class[i][indices_incorrect_class[i]]

In [10]:
models_ai = dict()                         
for k in range(5):
     i = k + 2
     models_ai[i] = load_model(os.path.join(models_path, 'xception_mags_'+str(i)+'.h5'))

In [11]:
for k in range(5):
    i = k + 2
    print("Num forces: ", i)
    loss = models_ai[i].evaluate(X[i][indices_correct_class[i], ], y_lai[i][indices_correct_class[i], ])

Num forces:  2
125/125 [==============================] - 36s 289ms/step - loss: 0.0070 - mean_absolute_error: 0.0070
Num forces:  3
125/125 [==============================] - 40s 318ms/step - loss: 0.0311 - mean_absolute_error: 0.0311
Num forces:  4
124/124 [==============================] - 36s 287ms/step - loss: 0.0552 - mean_absolute_error: 0.0552
Num forces:  5
121/121 [==============================] - 39s 319ms/step - loss: 0.0436 - mean_absolute_error: 0.0436
Num forces:  6
122/122 [==============================] - 3s 28ms/step - loss: 0.0481 - mean_absolute_error: 0.0481


In [12]:
diffs = []
for k in range(5):
    i = k + 2
    print("Num forces: ", i)
    if (len(indices_incorrect_class[i]) > 0):
        for j in range(len(indices_incorrect_class[i])):
            predicted_mags = models_ai[predicted_incorrect_class[i][j]].predict(X[i][None, indices_incorrect_class[i][j], ])
            true_mags = y_lai[i][indices_incorrect_class[i][j], ]
            diff = abs(np.sum(predicted_mags) - np.sum(true_mags))
            diffs.append(diff)
            print("Predicted: ", predicted_mags)
            print("True: ", true_mags)
            print("Difference: ", diff)      

Num forces:  2
Predicted:  [[0.05453008 0.0400901  0.04224825]]
True:  [0.01906226 0.01906226]
Difference:  0.09874391878146332
Predicted:  [[0.04623216 0.08559485 0.06820674]]
True:  [0.01906226 0.01906226]
Difference:  0.16190924072760743
Predicted:  [[0.05093007 0.05409314 0.05655453]]
True:  [0.01906226 0.01906226]
Difference:  0.12345321798819703
Predicted:  [[0.05807459 0.04045494 0.0393068 ]]
True:  [0.01906226 0.01906226]
Difference:  0.0997118237068097
Predicted:  [[0.04347056 0.04288603 0.03264754]]
True:  [0.01906226 0.01906226]
Difference:  0.08087961698073548
Predicted:  [[0.08592522 0.05857399 0.04061852]]
True:  [0.01906226 0.01906226]
Difference:  0.1469932230760495
Predicted:  [[0.04555872 0.03230806 0.04042035]]
True:  [0.01906226 0.01906226]
Difference:  0.08016261780757111
Predicted:  [[0.03233857 0.07020436 0.07191235]]
True:  [0.02018206 0.02018206]
Difference:  0.1340911625038618
Predicted:  [[0.02159604 0.01994516 0.01989941]]
True:  [0.02018206 0.02018206]
Diff

Predicted:  [[0.5259347  0.38194716 0.5318974 ]]
True:  [0.68051707 0.36984642 0.37474114 0.37573464]
Difference:  0.36105999292139845
Num forces:  5
Predicted:  [[0.10532916 0.09758428 0.1020785  0.09473974 0.09317767 0.1151268 ]]
True:  [0.10138878 0.12277444 0.09750693 0.13242641 0.19930396]
Difference:  0.04536436092969698
Predicted:  [[0.10528161 0.09551354 0.0967804  0.09445964 0.09354056 0.10932069]]
True:  [0.19930396 0.10138878 0.12277444 0.09750693 0.13242641]
Difference:  0.05850408566114229
Predicted:  [[0.09192645 0.10901718 0.11970778 0.09554102 0.08753948 0.13450398]]
True:  [0.19930396 0.10138878 0.12277444 0.09750693 0.13242641]
Difference:  0.015164654375284137
Predicted:  [[0.10724232 0.10560249 0.10428226 0.09395572 0.09028319 0.10966793]]
True:  [0.19930396 0.10138878 0.12277444 0.09750693 0.13242641]
Difference:  0.04236654532071871
Predicted:  [[0.11129062 0.1144269  0.12513037 0.10205404 0.10165521 0.13419244]]
True:  [0.12616048 0.113841   0.15103419 0.11799728

Predicted:  [[0.01607777 0.0152487  0.01433033 0.0142379 ]]
True:  [0.03385615 0.01331474 0.01646681 0.01968375 0.01826141]
Difference:  0.04168815133215656
Predicted:  [[0.03038627 0.02067193 0.0209027  0.02591692]]
True:  [0.03385615 0.01331474 0.01646681 0.01968375 0.01826141]
Difference:  0.003705024393813511
Predicted:  [[0.03060888 0.01953519 0.02606652 0.03190279]]
True:  [0.03385615 0.01331474 0.01646681 0.01968375 0.01826141]
Difference:  0.006530531076176266
Predicted:  [[0.05018046 0.04049662 0.05080057 0.05555631]]
True:  [0.03385615 0.01331474 0.01646681 0.01968375 0.01826141]
Difference:  0.09545110943673382
Predicted:  [[0.04994024 0.04296298 0.04965813 0.04896151]]
True:  [0.03385615 0.01331474 0.01646681 0.01968375 0.01826141]
Difference:  0.08994000437615643
Predicted:  [[0.0687755  0.06480055 0.07534204 0.0687054  0.05884054 0.0628179 ]]
True:  [0.05978533 0.10393431 0.08611521 0.07105284 0.11186049]
Difference:  0.033466229427948746
Predicted:  [[0.07906141 0.065893

Predicted:  [[0.24456687 0.2246236  0.26515806 0.13395682]]
True:  [0.06055889 0.20192133 0.16009907 0.2058238  0.18243352]
Difference:  0.057468718204790714
Predicted:  [[0.35632867 0.27704495 0.33817095 0.2816934 ]]
True:  [0.23952032 0.26904552 0.21593629 0.29503335 0.37897859]
Difference:  0.14527610695811255
Predicted:  [[0.323201   0.21407506 0.2730615  0.23959407 0.24362507 0.3101246 ]]
True:  [0.25493306 0.21150508 0.33238203 0.29054872 0.4271726 ]
Difference:  0.0871398314245051
Predicted:  [[0.80889654 0.7251484  0.7163458  0.7770393  0.61490583 0.7866877 ]]
True:  [0.74719487 0.77557583 0.6859697  0.37327616 0.96647637]
Difference:  0.8805308093666318
Predicted:  [[0.87362933 0.7247286  0.7077471  0.77419597 0.59384805 0.79595023]]
True:  [0.74719487 0.77557583 0.6859697  0.37327616 0.96647637]
Difference:  0.9216065158485653
Predicted:  [[0.7364773  0.54501337 0.54688865 0.61430794]]
True:  [0.80949951 0.62834254 0.46660681 0.47849634 0.44715885]
Difference:  0.387416771805

Predicted:  [[0.40147418 0.34743583 0.32008377 0.34769815 0.42007178]]
True:  [0.40080054 0.38630147 0.24691964 0.3362308  0.30206279 0.41882512]
Difference:  0.2543764940422695
Predicted:  [[0.40525302 0.37250164 0.3356532  0.3631069  0.4260416 ]]
True:  [0.40080054 0.38630147 0.24691964 0.3362308  0.30206279 0.41882512]
Difference:  0.1885839334648769
Predicted:  [[0.42148566 0.37434497 0.34701958 0.3641936  0.42477107]]
True:  [0.40080054 0.38630147 0.24691964 0.3362308  0.30206279 0.41882512]
Difference:  0.1593255630654018
Predicted:  [[0.33105904 0.30241442 0.2924208  0.31776726 0.3558693 ]]
True:  [0.3350206  0.28842696 0.22475725 0.24283155 0.27027136 0.28530535]
Difference:  0.047082249235888884
Predicted:  [[0.2702253  0.24443859 0.23643401 0.24769619 0.29421246]]
True:  [0.25804537 0.27662605 0.25259621 0.24000394 0.26453518 0.34223995]
Difference:  0.34104015906430885
Predicted:  [[0.3219694  0.32259846 0.29729968 0.35208002 0.32607275]]
True:  [0.37143781 0.27270686 0.2861

Predicted:  [[1.0290703  0.8502945  0.8561113  0.93610346 1.0042485 ]]
True:  [0.68370011 0.7612029  0.85937658 1.0516221  0.83475985 1.19735642]
Difference:  0.7121899708811776
Predicted:  [[1.0569313  0.8259021  0.84180135 0.8561071  1.0718552 ]]
True:  [1.19735642 0.68370011 0.7612029  0.85937658 1.0516221  0.83475985]
Difference:  0.7354210003916748
Predicted:  [[0.9769922  0.77227163 0.7827496  0.8776176  0.9240251 ]]
True:  [1.19735642 0.68370011 0.7612029  0.85937658 1.0516221  0.83475985]
Difference:  1.0543616398875244
Predicted:  [[1.2118969  0.8902266  0.90821415 1.0724459  1.0523164 ]]
True:  [1.19735642 0.68370011 0.7612029  0.85937658 1.0516221  0.83475985]
Difference:  0.25291758623762206
Predicted:  [[1.289993   0.83512056 0.8935343  1.049033   1.002326  ]]
True:  [1.19735642 0.68370011 0.7612029  0.85937658 1.0516221  0.83475985]
Difference:  0.31801110354108886
Predicted:  [[1.112963   0.8215076  0.8231903  0.97364223 0.97872096]]
True:  [1.19735642 0.68370011 0.76120

Predicted:  [[0.26231652 0.2250743  0.21901222 0.18988368 0.3017094 ]]
True:  [0.276269   0.0736892  0.19277494 0.2597067  0.22133745 0.38076494]
Difference:  0.2065460919008275
Predicted:  [[0.27816105 0.20523316 0.22006968 0.1923987  0.28390136]]
True:  [0.276269   0.0736892  0.19277494 0.2597067  0.22133745 0.38076494]
Difference:  0.22477831827259265
Predicted:  [[0.567301   0.43809316 0.46500444 0.48923406 0.48565573]]
True:  [0.2256368  0.49380982 0.39200175 0.45511679 0.34115595 0.61069972]
Difference:  0.07313240684753186
Predicted:  [[0.58591    0.3003226  0.38597584 0.36154103 0.41353726]]
True:  [0.61718663 0.37702869 0.27420926 0.39069195 0.3617499  0.35626451]
Difference:  0.32984419259145703
Predicted:  [[0.11940942 0.09910353 0.09980602 0.1048297  0.10158858]]
True:  [0.13829163 0.04429131 0.07314004 0.11125989 0.09239156 0.05541486]
Difference:  0.009947950031903852
Predicted:  [[0.11996476 0.1242771  0.11083914 0.10738406 0.10418751]]
True:  [0.13829163 0.04429131 0.07

In [13]:
print("Average difference: ", sum(diffs) / len(diffs))

Average difference:  0.21615821543155464
